In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import seaborn as sns

data_path = pathlib.Path().home()/'group'/'project'/'results'/'beamforming'\
/'time_dependent'/'beamform_signals'/'220307_0to5cm_84to90_8192_gradb_corrected.npz'

def interpolate(missing_inds, image):
    
    skipped_pixels = []
    
    for i in range(missing_inds.shape[0]):
        
        neighbor_x = []
        neighbor_y = []
        
        missing_y = missing_inds[i, 0]
        missing_x = missing_inds[i, 1]
        
        if missing_x + 1 < image.shape[1]:
            neighbor_x.append(missing_x + 1)
        if missing_x - 1 >= 0:
            neighbor_x.append(missing_x - 1)
        if missing_y + 1 < image.shape[0]:
            neighbor_y.append(missing_y + 1)
        if missing_y - 1 >= 0:
            neighbor_y.append(missing_y - 1)
            
        neighbor_x.append(missing_x)
        neighbor_y.append(missing_y)
            
        neighbor_grid_x, neighbor_grid_y = np.meshgrid(neighbor_x, neighbor_y)
        
        freq_list = []
        
        for j, pair in enumerate(zip(neighbor_grid_x.flatten(), neighbor_grid_y.flatten())):
            #print(pair, missing_inds[:, 0], missing_inds[:, 1])
            if not np.any(np.logical_and(missing_inds[:, 0] == pair[0], missing_inds[:, 1] == pair[1])):
                freq_list.append(image[pair[1], pair[0]])
                #print(freq_list)
                #input()
                
        if len(freq_list) == 0:
            skipped_pixels.append([missing_x, missing_y])
        else:
            image[missing_y, missing_x] = np.mean(freq_list)
    skipped_pixels = np.array(skipped_pixels)
    
    if skipped_pixels.shape[0] == 0:
        return image
    else:
        interpolate(skipped_pixels, image)

In [ ]:
data = np.load(data_path)
for i, key in enumerate(data): print(key)

In [ ]:
print(data['signals'].shape)

In [ ]:
r_grid, pa_grid = np.meshgrid(np.sort(np.unique(data['radius'])),
                              np.sort(np.unique(data['pitch'])))

missing_inds = []
signals_sort = np.zeros((r_grid.size, data['signals'].shape[-1]), np.complex128)
signals = data['signals'][:]
for i, pair in enumerate(zip(r_grid.flatten(), pa_grid.flatten())):
    
    #print(i)
    try:
        ind = np.argwhere(np.logical_and(
            data['radius'] == pair[0],
            data['pitch'] == pair[1]
        )).squeeze()
        signals_sort[i, :] = signals[ind, :]
        #print(i, ind)
    except:
        missing_inds.append(i)

In [ ]:
energy_grid = np.mean(abs(signals_sort) ** 2, axis=-1).reshape((r_grid.shape))

In [ ]:
fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
img = ax.imshow(
    energy_grid,
    aspect='auto',
    interpolation='none',
    extent = (0, 0.05, 84,90)
               )

cbar = fig.colorbar(img)

# Interpolate the missing pixels

In [ ]:
missing_inds = np.argwhere(energy_grid==0).squeeze()

In [ ]:
interpolated_energy_grid = np.array(energy_grid, copy=True)
interpolated_energy_grid = interpolate(missing_inds, energy_grid)


# plot the curves over the interpolated map

In [ ]:
curve_path = pathlib.Path.home()/'group'/'project'/'results'/'beamforming'/\
'power_map_detection_boundary'
name_static = '220309_static_bf_10db_threshold_boundary.npz'
name_bf = '220308_time_dependent_bf_10db_threshold_boundary.npz'
name_mf = '220308_mf_score2_threshold_boundary.npz'
name_dnn = '220308_time_dep_bf_dnn_detection_boundary.npz'

curve_static = np.load(curve_path/name_static)
curve_bf = np.load(curve_path/name_bf)
curve_mf = np.load(curve_path/name_mf)
curve_dnn = np.load(curve_path/name_dnn)

In [ ]:
sns.set_theme(context='talk', style='ticks')
cmap = sns.color_palette('plasma', as_cmap=True)
clist = sns.color_palette('bright', n_colors=15)

fig = plt.figure(figsize=(13, 8))
ax = fig.add_subplot(1,1,1)
img = ax.imshow(
    interpolated_energy_grid,
    aspect='auto',
    extent = (0, 0.05, 90,84),
    cmap=cmap
               )

cbar = fig.colorbar(img, label=r'$|V_{rms}^2|$')

#ax.plot(
#    np.concatenate((curve_static['radius'][1:], [0.023])), 
#    np.concatenate((curve_static['pitch'][1:], [90])),
#    color=clist[0], label='Static-BF', linewidth=4)
#ax.plot(curve_bf['radius'], curve_bf['pitch'], color=clist[2], label='BF', linewidth=4)
#ax.plot(curve_dnn['radius'], curve_dnn['pitch'], color=clist[9], label='DNN', linewidth=4)
#ax.plot(curve_mf['radius'], curve_mf['pitch'], color=clist[3], label='MF', linewidth=4)

#ax.legend(loc=1)
#ax.set_title('Estimated Detection Theshold Boundaries')

ax.set_ylabel('Pitch Angle (deg)')
ax.set_xlabel('Radius (m)')
plt.tight_layout()

save_path = pathlib.Path.home()/'group'/'project'/'plots'/'analysis'/'beamforming'\
/'power_maps'
#name='220318_static_vs_bf_vs_mf_vs_dnn_detection_threshold_electron_paramter_map'
name='220318_summed_power_electron_paramter_map'

#plt.savefig(save_path/name)

In [ ]:
np.concatenate((curve_static['radius'], [0.023]))

In [ ]:
max_inds = np.argmax(abs(np.fft.fft(signals_sort)), axis=-1)

In [ ]:


peak_grid = np.diag((abs(np.fft.fft(signals_sort) / 8192) ** 2)[:, max_inds])

In [ ]:
peak_grid.shape